# Combined  Model (XGBoost Undersampling + XGBoost Regression)


In [1]:
%load_ext jupyter_black

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import shap
import imblearn
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
from sklearn.dummy import DummyRegressor
from xgboost import XGBClassifier

# from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from matplotlib import cm

# from mlxtend.plotting import plot_confusion_matrix
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

from utils import get_training_dataset

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [3]:
# Read csv file and import to df
df = get_training_dataset()
df.head()

,typhoon_name,typhoon_year,grid_point_id,wind_speed,track_distance,rainfall_max_6h,rainfall_max_24h,total_houses,rwi,mean_slope,...,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged,percent_houses_damaged_5years
0,DURIAN,2006,101,0.0,303.180555,0.122917,0.085417,31.000000,NaN,1.018526,...,2.699781,5.762712,3445.709753,1,0.0,0.00,1.00,0.000000,0.0,0.0
1,DURIAN,2006,4475,0.0,638.027502,0.091667,0.027083,3.301020,-0.527000,1.579400,...,4.585088,12.799127,8602.645832,1,0.0,0.00,1.00,0.000000,0.0,0.0
2,DURIAN,2006,4639,0.0,603.631997,0.535417,0.146354,12.103741,-0.283000,0.551764,...,1.527495,8.833333,5084.012925,1,0.0,0.01,0.99,197.339034,0.0,0.0
3,DURIAN,2006,4640,0.0,614.675270,0.356250,0.101562,645.899660,-0.358889,2.107949,...,11.677657,17.530431,55607.865950,1,0.0,0.31,0.69,4970.477311,0.0,0.0
4,DURIAN,2006,4641,0.0,625.720905,0.202083,0.057812,1071.731293,-0.462800,3.538881,...,17.074011,31.931338,35529.342507,1,0.0,0.77,0.23,12408.594656,0.0,0.0


In [4]:
# Fill NaNs with average estimated value of 'rwi'
df["rwi"].fillna(df["rwi"].mean(), inplace=True)

# Set any values >100% to 100%,
for i in range(len(df)):
    if df.loc[i, "percent_houses_damaged"] > 100:
        df.at[i, "percent_houses_damaged"] = float(100)

In [5]:
# Remove zeros from wind_speed
df = (df[(df[["wind_speed"]] != 0).any(axis=1)]).reset_index(drop=True)
df = df.drop(columns=["grid_point_id", "typhoon_year"])
df.head()

,typhoon_name,wind_speed,track_distance,rainfall_max_6h,rainfall_max_24h,total_houses,rwi,mean_slope,std_slope,mean_tri,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged,percent_houses_damaged_5years
0,DURIAN,12.460039,275.018491,0.670833,0.313021,0.479848,-0.213039,12.896581,7.450346,74.625539,34.629550,42.218750,5303.659490,1,0.00,0.00,1.00,0.000000,0.0,0.0
1,DURIAN,11.428974,297.027578,0.929167,0.343229,55.649739,0.206000,14.070741,6.514647,68.681417,25.475388,72.283154,61015.543599,1,0.00,0.14,0.86,276.871504,0.0,0.0
2,DURIAN,13.077471,262.598363,0.716667,0.424479,8.157414,-0.636000,19.758682,10.940700,104.453163,54.353996,102.215198,66707.438070,1,0.00,0.11,0.89,448.539453,0.0,0.0
3,DURIAN,12.511864,273.639330,0.568750,0.336979,88.292015,-0.227500,11.499097,6.901584,59.798108,31.814048,58.988877,53841.050168,1,0.00,0.12,0.88,2101.708435,0.0,0.0
4,DURIAN,11.977511,284.680297,0.589583,0.290625,962.766739,-0.299667,13.866633,6.528689,65.655280,25.976413,111.386527,87378.257957,1,0.07,0.46,0.47,11632.726327,0.0,0.0


In [6]:
# Define bins for data stratification
bins2 = [0, 0.00009, 1, 10, 50, 101]
bins_eval = [0, 1, 10, 20, 50, 101]
samples_per_bin2, binsP2 = np.histogram(df["percent_houses_damaged"], bins=bins2)

In [7]:
# Check the bins' intervalls (first bin means all zeros, second bin means 0 < values <= 1)
df["percent_houses_damaged"].value_counts(bins=binsP2)

(-0.001, 9e-05]    38901
(9e-05, 1.0]        7232
(1.0, 10.0]         2552
(10.0, 50.0]         925
(50.0, 101.0]        144
Name: percent_houses_damaged, dtype: int64

In [8]:
print(samples_per_bin2)
print(binsP2)

[38901  7232  2552   925   144]
[0.00e+00 9.00e-05 1.00e+00 1.00e+01 5.00e+01 1.01e+02]


In [9]:
bin_index2 = np.digitize(df["percent_houses_damaged"], bins=binsP2)

In [10]:
y_input_strat = bin_index2

In [11]:
features = [
    "wind_speed",
    "track_distance",
    "total_houses",
    "rainfall_max_6h",
    "rainfall_max_24h",
    "rwi",
    "mean_slope",
    "std_slope",
    "mean_tri",
    "std_tri",
    "mean_elev",
    "coast_length",
    "with_coast",
    "urban",
    "rural",
    "water",
    "total_pop",
    "percent_houses_damaged_5years",
]

# Split X and y from dataframe features
X = df[features]
display(X.columns)
y = df["percent_houses_damaged"]

Index(['wind_speed', 'track_distance', 'total_houses', 'rainfall_max_6h',
       'rainfall_max_24h', 'rwi', 'mean_slope', 'std_slope', 'mean_tri',
       'std_tri', 'mean_elev', 'coast_length', 'with_coast', 'urban', 'rural',
       'water', 'total_pop', 'percent_houses_damaged_5years'],
      dtype='object')

In [12]:
# Define train and test data
X_train, X_test, y_train, y_test = train_test_split(
    X,
    df["percent_houses_damaged"],
    test_size=0.2,
    stratify=y_input_strat,
)

## First step is to train XGBoost Regression model for train data

In [13]:
# XGBoost Reduced Overfitting
xgb = XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=0.8,
    colsample_bynode=0.8,
    colsample_bytree=0.8,
    gamma=3,
    eta=0.01,
    importance_type="gain",
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=4,
    min_child_weight=1,
    missing=1,
    n_estimators=100,
    early_stopping_rounds=10,
    n_jobs=1,
    nthread=None,
    objective="reg:squarederror",
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=0.8,
    verbosity=1,
    eval_metric=["rmse", "logloss"],
    random_state=0,
)

eval_set = [(X_test, y_test)]
xgb_model = xgb.fit(X_train, y_train, eval_set=eval_set, verbose=False)

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[15:33:02] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [14]:
# Make prediction on train and test data
y_pred_train = xgb.predict(X_train)
y_pred = xgb.predict(X_test)

In [15]:
# Calculate RMSE in total

mse_train_idx = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train_idx)

mse_idx = mean_squared_error(y_test, y_pred)
rmseM1 = np.sqrt(mse_idx)

print(f"RMSE_test_in_total: {rmseM1:.2f}")
print(f"RMSE_train_in_total: {rmse_train:.2f}")

RMSE_test_in_total: 3.32
RMSE_train_in_total: 2.60


In [16]:
# Calculate RMSE per bins

bin_index_test = np.digitize(y_test, bins=bins_eval)
bin_index_train = np.digitize(y_train, bins=bins_eval)

RSME_test_model1 = np.zeros(len(bins_eval) - 1)

for bin_num in range(1, len(bins_eval)):

    # Estimation of RMSE for train data
    mse_train_idx = mean_squared_error(
        y_train[bin_index_train == bin_num], y_pred_train[bin_index_train == bin_num]
    )
    rmse_train = np.sqrt(mse_train_idx)

    # Estimation of RMSE for test data
    mse_idx = mean_squared_error(
        y_test[bin_index_test == bin_num], y_pred[bin_index_test == bin_num]
    )
    RSME_test_model1[bin_num - 1] = np.sqrt(mse_idx)

    print(
        f"RMSE_test  [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {RSME_test_model1[bin_num-1]:.2f}"
    )
    print(
        f"RMSE_train [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {rmse_train:.2f}"
    )

RMSE_test  [0,1): 1.05
RMSE_train [0,1): 0.91
RMSE_test  [1,10): 4.56
RMSE_train [1,10): 3.88
RMSE_test  [10,20): 9.74
RMSE_train [10,20): 8.93
RMSE_test  [20,50): 19.42
RMSE_train [20,50): 16.01
RMSE_test  [50,101): 40.34
RMSE_train [50,101): 27.23


## Second step is to train XGBoost Binary model for same train data

In [17]:
# Define a threshold to separate target into damaged and not_damaged
thres = 10.0
y_test_bool = y_test >= thres
y_train_bool = y_train >= thres
y_test_bin = (y_test_bool) * 1
y_train_bin = (y_train_bool) * 1

In [18]:
sum(y_train_bin)

855

In [19]:
print(Counter(y_train_bin))

Counter({0: 38948, 1: 855})


In [20]:
# Undersampling

# Define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.1)
# Fit and apply the transform
X_train_us, y_train_us = under.fit_resample(X_train, y_train_bin)

print(Counter(y_train_us))

Counter({0: 8550, 1: 855})


In [21]:
# Use XGBClassifier as a Machine Learning model to fit the data
xgb_model = XGBClassifier(eval_metric=["error", "logloss"])

# eval_set = [(X_train, y_train), (X_train, y_train)]
eval_set = [(X_test, y_test_bin)]
xgb_model.fit(
    X_train_us,
    y_train_us,
    eval_set=eval_set,
    verbose=False,
)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric=['error', 'logloss'], gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [22]:
# Make prediction on test data
y_pred_test = xgb_model.predict(X_test)

In [23]:
# Print Confusion Matrix
cm = confusion_matrix(y_test_bin, y_pred_test)
cm

array([[9589,  148],
       [  49,  165]])

In [24]:
# Classification Report
print(metrics.classification_report(y_test_bin, y_pred_test))
print(metrics.confusion_matrix(y_test_bin, y_pred_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      9737
           1       0.53      0.77      0.63       214

    accuracy                           0.98      9951
   macro avg       0.76      0.88      0.81      9951
weighted avg       0.98      0.98      0.98      9951

[[9589  148]
 [  49  165]]


In [25]:
# Make prediction on train data
y_pred_train = xgb_model.predict(X_train)

In [26]:
# Print Confusion Matrix
cm = confusion_matrix(y_train_bin, y_pred_train)
cm

array([[38507,   441],
       [    0,   855]])

In [27]:
# Classification Report
print(metrics.classification_report(y_train_bin, y_pred_train))
print(metrics.confusion_matrix(y_train_bin, y_pred_train))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     38948
           1       0.66      1.00      0.79       855

    accuracy                           0.99     39803
   macro avg       0.83      0.99      0.89     39803
weighted avg       0.99      0.99      0.99     39803

[[38507   441]
 [    0   855]]


In [28]:
reduced_df = X_train.copy()

In [29]:
reduced_df["percent_houses_damaged"] = y_train.values
reduced_df["predicted_value"] = y_pred_train

In [30]:
fliterd_df = reduced_df[reduced_df.predicted_value == 1]

In [31]:
fliterd_df

,wind_speed,track_distance,total_houses,rainfall_max_6h,rainfall_max_24h,rwi,mean_slope,std_slope,mean_tri,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged_5years,percent_houses_damaged,predicted_value
1064,66.866163,9.240434,23953.805337,10.810417,5.465625,0.141200,3.340877,4.506943,16.314108,20.168021,64.568955,20095.546554,1,0.46,0.22,0.32,132622.070162,0.000000,47.143696,1
1021,64.448368,22.144369,22008.718478,12.281250,6.524479,0.164520,3.286739,4.357487,16.838199,19.288742,80.300253,0.000000,0,0.65,0.35,0.00,120945.373232,0.000000,32.412728,1
26059,59.661793,17.012016,2734.106500,9.693750,4.775521,0.291750,7.148000,6.446785,35.722194,28.628319,59.999788,42000.379821,1,0.10,0.55,0.35,22319.321347,0.087531,0.634468,1
31545,49.558243,40.254697,82.928836,17.616667,8.138021,-0.744000,13.340749,7.569959,64.199679,31.687829,178.685364,20402.634968,1,0.00,0.73,0.27,87.338798,0.000000,18.730159,1
11752,60.757570,5.611486,126.013788,12.860417,5.148958,-0.127333,12.195858,5.770609,60.491933,22.031380,57.004651,17034.871570,1,0.00,0.07,0.93,1644.562218,0.000000,32.772631,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25065,57.701218,14.750272,5821.272467,16.389583,7.548958,-0.252700,2.032354,1.982336,11.810215,9.051913,20.431010,33421.504753,1,0.22,0.18,0.60,32043.365086,0.195289,22.380168,1
39759,51.995493,42.389185,2269.299399,10.470833,8.072396,-0.237000,8.990046,6.061984,44.341444,26.881107,150.877699,33143.232571,1,0.15,0.35,0.50,21033.027965,0.507295,16.667566,1
13009,71.578411,1.977647,730.146441,11.856250,5.760417,-0.439286,14.625203,8.273153,71.197216,38.148047,632.592787,0.000000,0,0.00,1.00,0.00,4457.891940,0.000264,51.840843,1
18584,56.426904,24.318503,5221.600451,13.504167,5.831250,-0.308080,4.761031,3.687369,24.959946,15.480483,84.557407,0.000000,0,0.14,0.86,0.00,32897.048897,0.000550,5.564521,1


### Third step is to train XGBoost regression model for this reduced train data (including damg>10.0%)

In [32]:
# Define bins for data stratification in regression model
bins2 = [0, 1, 10, 20, 50, 101]
samples_per_bin2, binsP2 = np.histogram(
    fliterd_df["percent_houses_damaged"], bins=bins2
)

print(samples_per_bin2)
print(binsP2)

[184 257 381 359 115]
[  0   1  10  20  50 101]


In [33]:
bin_index2 = np.digitize(fliterd_df["percent_houses_damaged"], bins=binsP2)

In [34]:
y_input_strat = bin_index2

In [35]:
# Split X and y from dataframe features
X_r = fliterd_df[features]
display(X.columns)
y_r = fliterd_df["percent_houses_damaged"]

Index(['wind_speed', 'track_distance', 'total_houses', 'rainfall_max_6h',
       'rainfall_max_24h', 'rwi', 'mean_slope', 'std_slope', 'mean_tri',
       'std_tri', 'mean_elev', 'coast_length', 'with_coast', 'urban', 'rural',
       'water', 'total_pop', 'percent_houses_damaged_5years'],
      dtype='object')

In [36]:
# XGBoost Reduced Overfitting
xgbR = XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=0.8,
    colsample_bynode=0.8,
    colsample_bytree=0.8,
    gamma=3,
    eta=0.01,
    importance_type="gain",
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=4,
    min_child_weight=1,
    missing=1,
    n_estimators=100,
    early_stopping_rounds=10,
    n_jobs=1,
    nthread=None,
    objective="reg:squarederror",
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=0.8,
    verbosity=1,
    eval_metric=["rmse", "logloss"],
    random_state=0,
)

eval_set = [(X_r, y_r)]
xgbR_model = xgbR.fit(X_r, y_r, eval_set=eval_set, verbose=False)

[15:36:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [37]:
# Make prediction on train and global test data
y_pred_r = xgbR.predict(X_r)
y_pred_test_total = xgbR.predict(X_test)

In [38]:
# Calculate RMSE in total

mse_train_idxR = mean_squared_error(y_r, y_pred_r)
rmse_trainR = np.sqrt(mse_train_idxR)


mse_idxR = mean_squared_error(y_test, y_pred_test_total)
rmseR = np.sqrt(mse_idxR)

print(f"RMSE_test_in_total MR: {rmseR:.2f}")
print(f"RMSE_test_in_total M1: {rmseM1:.2f}")
print(f"RMSE_train_in_reduced: {rmse_trainR:.2f}")

RMSE_test_in_total MR: 17.79
RMSE_test_in_total M1: 3.32
RMSE_train_in_reduced: 9.71


In [39]:
# Calculate RMSE per bins
bin_index_r = np.digitize(y_r, bins=bins_eval)

RSME_test_model1R = np.zeros(len(bins_eval) - 1)
for bin_num in range(1, len(bins_eval)):

    # Estimation of RMSE for train data
    mse_train_idxR = mean_squared_error(
        y_r[bin_index_r == bin_num], y_pred_r[bin_index_r == bin_num]
    )
    rmse_trainR = np.sqrt(mse_train_idxR)

    # Estimation of RMSE for test data
    mse_idxR = mean_squared_error(
        y_test[bin_index_test == bin_num], y_pred_test_total[bin_index_test == bin_num]
    )
    RSME_test_model1R[bin_num - 1] = np.sqrt(mse_idxR)

    # print(f"RMSE_test: {rmse:.2f}")
    print(
        f"RMSE_train_reduced [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {rmse_trainR:.2f}"
    )
    print(
        f"RMSE_test_total_MR [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {RSME_test_model1R[bin_num-1]:.2f}"
    )
    print(
        f"RMSE_test_total_M1 [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {RSME_test_model1[bin_num-1]:.2f}"
    )
    RSME_test_model1
    # print(f"RMSE_train: {rmse_train:.2f}")

RMSE_train_reduced [0,1): 10.87
RMSE_test_total_MR [0,1): 18.02
RMSE_test_total_M1 [0,1): 1.05
RMSE_train_reduced [1,10): 8.07
RMSE_test_total_MR [1,10): 13.13
RMSE_test_total_M1 [1,10): 4.56
RMSE_train_reduced [10,20): 4.70
RMSE_test_total_MR [10,20): 6.95
RMSE_test_total_M1 [10,20): 9.74
RMSE_train_reduced [20,50): 9.77
RMSE_test_total_MR [20,50): 14.40
RMSE_test_total_M1 [20,50): 19.42
RMSE_train_reduced [50,101): 18.88
RMSE_test_total_MR [50,101): 38.12
RMSE_test_total_M1 [50,101): 40.34


## Fourth step is to add model combination (model M1 with model MR)

In [40]:
# Check the result of classifier for test set
reduced_test_df = X_test.copy()

In [41]:
# joined X_test with countinous target and binary predicted values
reduced_test_df["percent_houses_damaged"] = y_test.values
reduced_test_df["predicted_value"] = y_pred_test

reduced_test_df

,wind_speed,track_distance,total_houses,rainfall_max_6h,rainfall_max_24h,rwi,mean_slope,std_slope,mean_tri,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged_5years,percent_houses_damaged,predicted_value
1458,34.327614,33.806625,8401.903069,2.945833,1.252083,0.183533,1.629498,1.238575,9.262738,5.820439,27.624873,52976.820734,1,0.400000,0.140000,0.46,58780.722564,0.000000,0.000000,0
27579,12.469671,259.365685,2640.866207,5.037500,3.558854,-0.183308,5.651087,4.852481,28.597375,20.996346,42.838199,44945.005679,1,0.060000,0.480000,0.46,12082.959505,0.102976,0.000000,0
27048,22.021167,43.401810,14.697209,5.900000,4.018750,-0.707000,23.325802,8.327006,113.383990,35.533915,842.405368,0.000000,0,0.000000,1.000000,0.00,1302.927877,0.140222,0.000000,0
28971,6.343438,266.879049,25788.770206,1.897917,0.798958,0.359280,0.601147,0.378647,4.468070,1.985849,20.449286,0.000000,0,0.750000,0.250000,0.00,140961.494394,0.000671,0.000000,0
33929,7.557865,280.908427,1219.343973,3.350000,2.075521,-0.427182,19.906252,11.246073,98.870945,65.456162,1354.304214,0.000000,0,0.045455,0.954545,0.00,8662.279183,0.347648,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,18.367882,187.696434,2917.678853,1.385417,0.783333,0.325286,6.485259,6.516899,29.450786,26.833987,85.754462,11790.682315,1,0.170000,0.130000,0.70,12075.830069,0.000000,0.000000,0
44144,19.224860,47.145049,3057.791880,6.560417,2.496354,-0.425680,2.467111,1.899664,14.311390,8.424930,31.519243,0.000000,0,0.150000,0.850000,0.00,19533.455747,0.309438,2.021853,0
19060,21.526785,136.487884,3900.686180,6.770833,4.820833,-0.180375,5.755625,6.066515,28.857457,26.302937,41.319990,80273.302342,1,0.200000,0.180000,0.62,22603.418101,0.079413,3.326762,0
15419,14.482464,37.809763,534.290564,6.020833,4.795833,0.000000,0.971760,0.590260,6.490326,3.176415,7.302885,4283.982773,1,0.010000,0.020000,0.97,865.582723,0.000000,0.000000,0


In [42]:
# damaged prediction
fliterd_test_df1 = reduced_test_df[reduced_test_df.predicted_value == 1]

# not damaged prediction
fliterd_test_df0 = reduced_test_df[reduced_test_df.predicted_value == 0]

In [43]:
# Use X0 and X1 for the M1 and MR models' predictions
X1 = fliterd_test_df1[features]
X0 = fliterd_test_df0[features]

In [44]:
# For the output equal to 1 apply MR to evaluate the performance
y1_pred = xgbR.predict(X1)
y1 = fliterd_test_df1["percent_houses_damaged"]

In [45]:
# For the output equal to 0 apply M1 to evaluate the performance
y0_pred = xgb.predict(X0)
y0 = fliterd_test_df0["percent_houses_damaged"]

In [46]:
# Calculate RMSE in total

mse_idxR_X0 = mean_squared_error(y0, y0_pred)
rmseR_X0 = np.sqrt(mse_idxR_X0)

mse_idxR_X1 = mean_squared_error(y1, y1_pred)
rmseR_X1 = np.sqrt(mse_idxR_X1)

print(f"RMSE_test_in_total M1: {rmseR_X0:.2f}")
print(f"RMSE_test_in_total MR: {rmseR_X1:.2f}")

RMSE_test_in_total M1: 1.50
RMSE_test_in_total MR: 16.58


In [47]:
bin_index_test1 = np.digitize(y1, bins=bins_eval)
bin_index_test0 = np.digitize(y0, bins=bins_eval)

RSME_y1_MR = np.zeros(len(bins_eval) - 1)
RSME_y0_M1 = np.zeros(len(bins_eval) - 1)

for bin_num in range(1, len(bins_eval)):

    if bin_num < 3:  # Model M1
        mse_idx1 = mean_squared_error(
            y0[bin_index_test0 == bin_num], y0_pred[bin_index_test0 == bin_num]
        )
        RSME_y0_M1[bin_num - 1] = np.sqrt(mse_idx1)

        print(
            f"RMSE_test estimated by Model M1 [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {RSME_y0_M1[bin_num-1]:.2f}"
        )

    else:  # Model MR

        mse_idxR = mean_squared_error(
            y1[bin_index_test1 == bin_num], y1_pred[bin_index_test1 == bin_num]
        )
        RSME_y1_MR[bin_num - 1] = np.sqrt(mse_idxR)

        print(
            f"RMSE_test estimated by Model MR [{bins_eval[bin_num-1]:.0f},{bins_eval[bin_num]:.0f}): {RSME_y1_MR[bin_num-1]:.2f}"
        )

RMSE_test estimated by Model M1 [0,1): 0.72
RMSE_test estimated by Model M1 [1,10): 2.94
RMSE_test estimated by Model MR [10,20): 7.80
RMSE_test estimated by Model MR [20,50): 14.71
RMSE_test estimated by Model MR [50,101): 37.86
